In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/open-shopee-code-league-marketing-analytics/users.csv
/kaggle/input/open-shopee-code-league-marketing-analytics/test.csv
/kaggle/input/open-shopee-code-league-marketing-analytics/sample_submission_0_1.csv
/kaggle/input/open-shopee-code-league-marketing-analytics/train.csv


# Reading Data

In [2]:
train = pd.read_csv("../input/open-shopee-code-league-marketing-analytics/train.csv")
test = pd.read_csv("../input/open-shopee-code-league-marketing-analytics/test.csv")
users = pd.read_csv("../input/open-shopee-code-league-marketing-analytics/users.csv")

# EDA

In [3]:
train.describe()

,country_code,user_id,subject_line_length,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id
count,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.000000,73539.00000
mean,2.533159,63048.595358,43.656373,0.911829,2.780457,5.019364,8.382287,25.284053,49.010430,0.909898,2.796040,5.420960,0.155781,36769.00000
std,1.675251,36068.006037,11.219323,1.754052,4.570239,7.902313,13.424436,37.853189,71.819327,2.887416,7.686064,13.980182,0.362650,21229.02506
min,1.000000,2.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.000000,31679.500000,34.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,18384.50000
50%,2.000000,63340.000000,43.000000,0.000000,1.000000,2.000000,3.000000,11.000000,22.000000,0.000000,0.000000,1.000000,0.000000,36769.00000
75%,3.000000,94731.500000,52.000000,1.000000,3.000000,6.000000,12.000000,35.000000,69.000000,1.000000,3.000000,6.000000,0.000000,55153.50000
max,7.000000,127925.000000,75.000000,31.000000,92.000000,135.000000,254.000000,735.000000,1260.000000,157.000000,531.000000,783.000000,1.000000,73538.00000


# Data Preparation

In [4]:
def data_cleaning(df):
    
    df.last_open_day[df.last_open_day == 'Never open'] = np.nan
    df.last_login_day[df.last_login_day == 'Never login'] = np.nan
    df.last_checkout_day[df.last_checkout_day == 'Never checkout'] = np.nan
    
    # Converting date column to day format
    df['grass_date'] = pd.to_datetime(df['grass_date']).apply(lambda x: x.weekday())
    
    # Replacing NaN with real values
    df.fillna({"last_open_day":df['last_open_day'].dropna().max(),
                 "last_login_day":df['last_login_day'].dropna().max(),
                 "last_checkout_day":df['last_checkout_day'].dropna().max(),
                 "attr_1": 2,
                 "attr_2": 2,
                 "attr_3": 2,
                 "age": df['age'].median()}, inplace = True)
    
    # One-hot encoding country code and domain column
    df = pd.concat([df.drop('country_code', axis=1), pd.get_dummies(df['country_code'])], axis=1)
    df = pd.concat([df.drop('domain', axis=1), pd.get_dummies(df['domain'])], axis=1)
    
    return df

In [5]:
# Pre-processing train set
train = train.merge(users, how="inner")
train = data_cleaning(train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [6]:
# Dropping unrevelant columns to be used as dataset for train-test split for X
X = train.drop(columns = ['user_id',
                          'row_id',
                          'open_flag',
                         ])
# y variable
y = train.open_flag

# Model definition and fit

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 12345)

In [8]:
from sklearn.ensemble import RandomForestClassifier

# Model defined and fit
ranfor = RandomForestClassifier(max_depth = 500, n_estimators = 500, n_jobs = -1, bootstrap = True)
rf = ranfor.fit(X_train, y_train)

# Getting predictions for X_test
Y_pred = ranfor.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score, matthews_corrcoef

def evaluate_model(model, X_valid, y_valid, metric=accuracy_score):
    y_pred = model.predict(X_valid)
    return metric(y_valid, y_pred)

# Evaluate model
accuracy_score = evaluate_model(ranfor, X_test, y_test, metric=accuracy_score)
matthews_score = evaluate_model(ranfor, X_test, y_test, metric=matthews_corrcoef)
print(f'Accuracy Score = {accuracy_score}')
print(f'MCC Score = {matthews_score}')

Accuracy Score = 0.8882920859396247
MCC Score = 0.5094314390442157


# Getting Test predictions

In [10]:
# Pre-processing Test data
test = test.merge(users, how="inner")
test = data_cleaning(test)

test_pred = test.drop(columns = ['user_id','row_id'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [11]:
# Getting predictions for Test
ans_pred = ranfor.predict(test_pred)

# Output to .csv

In [12]:
# Formatting submission.csv 
test['open_flag'] = -1
ans = test[['row_id','open_flag']]
ans['open_flag'] = ans_pred

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
# Output file
ans.to_csv('submission.csv', index=False)

In [14]:
ans

,row_id,open_flag
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
55965,55965,0
55966,55966,0
55967,55967,0
55968,55968,0
